1/14 최초 구현 by 소연 

수정 및 테스트 시 본 파일이 아닌 사본 사용을 부탁드립니다.

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/KPMG_Ideation
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pprint import pprint
from krwordrank.word import KRWordRank
from copy import deepcopy
import kss
import itertools
import unicodedata
import requests
from functools import reduce
from bs4 import BeautifulSoup
import string
import torch
from textrankr import TextRank
from lexrankr import LexRank
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
from pydub import AudioSegment
from konlpy.tag import Okt
import re
import nltk
# nltk.download('punkt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/KPMG_Ideation


In [ ]:
# import pre-trained model -- frameBERT (pytorch GPU 환경 필요)
%cd /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT
!pip install transformers 
import frame_parser
path="/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT"
parser = frame_parser.FrameParser(model_path=path, language='ko')

/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT
srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pretrained BERT: bert-base-multilingual-cased
using TGT special token: True
used dictionary:
	 /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lu2idx.json
	 /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lufrmap.json
	 /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/mul_bio_frargmap.json
...loaded model path: /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT
/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT
...model is loaded


In [ ]:
##### below are permanently installed packages #####
# nb_path = '/content/notebooks'
# os.symlink('/content/drive/Shareddrives/KPMG_Ideation', nb_path)
# sys.path.insert(0, nb_path)
# !pip install --target=$nb_path pydub
# !pip install --target=$nb_path kss
# %cd /content/drive/Shareddrives/KPMG_Ideation/hanspell
# !python setup.py install
# !pip install --target=$nb_path transformers
# !apt-get update
# !apt-get g++ openjdk-8-jdk 
# !pip3 install --target=$nb_path konlpy
# !pip install --target=$nb_path soykeyword
# !pip install --target=$nb_path krwordrank
# !pip install --target=$nb_path bert
# !pip install --target=$nb_path textrankr
# !pip install --target=$nb_path lexrankr

In [ ]:
# Due to google api credentials, SpeechRecognition needs to be installed everytime
!pip install SpeechRecognition
import speech_recognition as sr
# !pip install --upgrade google-cloud-speech

In [ ]:
def to_wav(audio_file_name):
    if audio_file_name.split('.')[1] == 'mp3':    
        sound = AudioSegment.from_mp3(audio_file_name)
        audio_file_name = audio_file_name.split('.')[0] + '.wav'
        sound.export(audio_file_name, format="wav")
    if audio_file_name.split('.')[1] == 'm4a':
        sound = AudioSegment.from_file(file_name,'m4a')
        audio_file_name = audio_file_name.replace('m4a','wav')
        sound.export(audio_file_name, format="wav")

In [ ]:
#!/usr/bin/env python3
files_path = ''
file_name = ''

startMin = 0
startSec = 0

endMin = 4
endSec = 30

# Time to miliseconds
startTime = startMin*60*1000+startSec*1000
endTime = endMin*60*1000+endSec*1000

%cd /content/drive/Shareddrives/KPMG_Ideation/data
file_name='audio_only_1.m4a'
track = AudioSegment.from_file(file_name,'m4a')
wav_filename = file_name.replace('m4a', 'wav')
file_handle = track.export(wav_filename, format='wav')
song = AudioSegment.from_wav('audio_only_1.wav')
extract = song[startTime:endTime]

# Saving as wav
extract.export('result.wav', format="wav")

AUDIO_FILE = os.path.join(os.path.dirname(os.path.abspath('data')), "result.wav")

# use the audio file as the audio source
r = sr.Recognizer()
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)  # read the entire audio file

# recognize speech using Google Speech Recognition
try:
    # for testing purposes, we're just using the default API key
    # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    # instead of `r.recognize_google(audio)`
    txt = r.recognize_google(audio, language='ko')
    print("Google Speech Recognition:" + txt)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))
    

/content/drive/Shareddrives/KPMG_Ideation/data
Google Speech Recognition:문제가 있었다 그 알고리즘으로 막상 해 놨길래 했는데 거기서 그 의료인들이 거의 다 빠지고 1순위에서 의료인들이 제일 필요한데 개를 다 빠지고 다른데 접종을 하는 것 때문에 막 난리가 났다고 그래가지고 알고리즘을 다시 만나기로 했어 나 뭐 이런 얘기 하다 그래서 거기서 어떻게 그 어떤 자료들을 사용을 한다면은 아까 말씀했듯이 그 바이러스 노출도 직업에따른 바이러스 노출도 그러나이 그걸 가진 크게 두 개로 받고 멜론에서 나이가 65세 이상 해결하는 거 아닌가 25세 이하 라던가 하면은 그 조금 더 가산점을 주는 거라던가 아니면은 뭐 직업특성상 뭐 그 의도가 이런데 종사하면 가산점을 주는 거고 근데 반면에 이제 조금 문제가 어떻게 될 수 있냐면 직업특성 만 가지고 하면 내 막상 재택근무자 아들이라도 산업군이 막 그런 그쪽에서 켜면은 태권도는 아무런 위험에 노출되어 있지 않은데 그런 식으로 바다쪽으로 받아가지고 오히려 더 그런 그런 문제들이 아직 해결 할 때 많이 남았다고 찾아왔어요 굉장히 좋은 지적이십니다 그렇다면 직업 특성상 재택근무가 아예 불가능한 집중된 섞여 있을 거 같은데 직업특성 뿐만이 아니라 그냥 바이러스 노출도 제일 중요한 거 같기도 하고 그 그게 이제 목적에 따라서 누굴먼저 맞춰야 되니까 조금 다른게 그 감염률을 나 죽고 싶으면네 사실 뭐랄까 제일 활발하게 활동하는 새끼들을 먼저 맞춰야지 그래야지 지금 뭐 그 퍼트리고 다니는 애들이 안 걸려 있으니까 그래서 그런 경우 조금 야외활동이 라던가 이런 것들이 활동이 많은 젊은 층들이 맞춰야 되는 반면에 치사율을 낮추고자를 목표로 하면은 우선은 노인 인구가 가장 큰 높으니까 애들 먼저 맞춰야 된다 이런 일이 있어가지고 달달 말씀이시죠 목적에 따라 달라진다 목적에 따라 준다 그리고 또 다른 이슈가 될 만한게 또 배탈 개 중에 하나가 박신 백신을 맞고 나서 이제 어느 정도로 확진자 

In [ ]:
%cd /content/drive/Shareddrives/KPMG_Ideation/hanspell
from hanspell import spell_checker
chked=""
line = kss.split_sentences(txt)
for i in range(len(line)):
  line[i] = spell_checker.check(line[i])[2]
  print("Checked spelling ",line[i])
  chked += "".join(line[i])
  chked += ". "

/content/drive/Shareddrives/KPMG_Ideation/hanspell
Checked spelling  문제가 있었다
Checked spelling  그 알고리즘으로 막상 해 놨길래 했는데 거기서 그 의료인들이 거의 다 빠지고 1순위에서 의료인들이 제일 필요한데 개를 다 빠지고 다른데 접종을 하는 것 때문에 막 난리가 났다고 그래가지고 알고리즘을 다시 만나기로 했어 나 뭐 이런 얘기 하다
Checked spelling  그래서 거기서 어떻게 그 어떤 자료들을 사용을 한다면은 아까 말씀했듯이 그 바이러스 노출도 직업에 따른 바이러스 노출도 그러나 이 그걸 가진 크게 두 개로 받고 멜론에서 나이가 65세 이상 해결하는 거 아닌가 25세 이하라던가 하면은 그 조금 더 가산점을 주는 거라던가 아니면은 뭐 직업 특성상 뭐 그 의도가 이런데 종사하면 가산점을 주는 거고 근데 반면에 이제 조금 문제가 어떻게 될 수 있냐면 직업 특성 만 가지고 하면 내 막상 재택근무자 아들이라도 산업 군이 막 그런 그쪽에서 켜면은 태권도는 아무런 위험에 노출되어 있지 않은데 그런 식으로 바다 쪽으로 받아가지고 오히려 더 그런 그런 문제들이 아직 해결할 때 많이 남았다고 찾아왔어요
Checked spelling  굉장히 좋은 지적이십니다
Checked spelling  그렇다면 직업 특성상 재택근무가 아예 불가능한 집중된 섞여 있을 거 같은데 직업 특성뿐만이 아니라 그냥 바이러스 노출도 제일 중요한 거 같기도 하고 그 그게 이제 목적에 따라서 누굴 먼저 맞춰야 되니까 조금 다른게 그 감염률을 나 죽고 싶으면 네 사실 뭐랄까 제일 활발하게 활동하는 새끼들을 먼저 맞춰야지 그래야지 지금 뭐 그 퍼트리고 다니는 애들이 안 걸려 있으니까 그래서 그런 경우 조금 야외활동이라던가 이런 것들이 활동이 많은 젊은 층들이 맞춰야 되는 반면에 치사율을 낮추고자를 목표로 하면은 우선은 노인 인구가 가장 큰 높으니까 애들 먼저 맞춰야 된다 이런 일이 있어가지고 달달 말씀이시죠 목적에 따라 달라

In [ ]:
chked

'문제가 있었다. 그 알고리즘으로 막상 해 놨길래 했는데 거기서 그 의료인들이 거의 다 빠지고 1순위에서 의료인들이 제일 필요한데 개를 다 빠지고 다른데 접종을 하는 것 때문에 막 난리가 났다고 그래가지고 알고리즘을 다시 만나기로 했어 나 뭐 이런 얘기 하다. 그래서 거기서 어떻게 그 어떤 자료들을 사용을 한다면은 아까 말씀했듯이 그 바이러스 노출도 직업에 따른 바이러스 노출도 그러나 이 그걸 가진 크게 두 개로 받고 멜론에서 나이가 65세 이상 해결하는 거 아닌가 25세 이하라던가 하면은 그 조금 더 가산점을 주는 거라던가 아니면은 뭐 직업 특성상 뭐 그 의도가 이런데 종사하면 가산점을 주는 거고 근데 반면에 이제 조금 문제가 어떻게 될 수 있냐면 직업 특성 만 가지고 하면 내 막상 재택근무자 아들이라도 산업 군이 막 그런 그쪽에서 켜면은 태권도는 아무런 위험에 노출되어 있지 않은데 그런 식으로 바다 쪽으로 받아가지고 오히려 더 그런 그런 문제들이 아직 해결할 때 많이 남았다고 찾아왔어요. 굉장히 좋은 지적이십니다. 그렇다면 직업 특성상 재택근무가 아예 불가능한 집중된 섞여 있을 거 같은데 직업 특성뿐만이 아니라 그냥 바이러스 노출도 제일 중요한 거 같기도 하고 그 그게 이제 목적에 따라서 누굴 먼저 맞춰야 되니까 조금 다른게 그 감염률을 나 죽고 싶으면 네 사실 뭐랄까 제일 활발하게 활동하는 새끼들을 먼저 맞춰야지 그래야지 지금 뭐 그 퍼트리고 다니는 애들이 안 걸려 있으니까 그래서 그런 경우 조금 야외활동이라던가 이런 것들이 활동이 많은 젊은 층들이 맞춰야 되는 반면에 치사율을 낮추고자를 목표로 하면은 우선은 노인 인구가 가장 큰 높으니까 애들 먼저 맞춰야 된다 이런 일이 있어가지고 달달 말씀이시죠 목적에 따라 달라진다. 목적에 따라 준다. 그리고 또 다른 이슈가 될 만한 게 또 배탈 개 중에 하나가 박신 백신을 맞고 나서 이제 어느 정도로 확진자 수가 누적돼서 증가하는 야 그걸 또 어느 도착할 것으로 보는 경우도 많잖아요. 그러려면 그렇죠 그런데 그럴 

In [ ]:
okt = Okt()
class Text():
    def __init__(self, text):
        text = re.sub("'", ' ', text)
        paragraphs = text.split('\n')
        self.text = text
        self.paragraphs = [i for i in paragraphs if i]
        self.counts = len(self.paragraphs)
        self.docs = [kss.split_sentences(paragraph) for paragraph in paragraphs if kss.split_sentences(paragraph)]
        self.newtext = deepcopy(self.text)
        print("TEXT")

    def findall(self, p, s):
        i = s.find(p)
        while i != -1:
            yield i
            i = s.find(p, i + 1)
      
    def countMatcher(self, sentences, paragraph_no):
        paragraph = self.docs[paragraph_no]
        total_no = len(paragraph)
        vec = [0] * total_no
        
        for idx, candidate in enumerate(paragraph):
            for sentence in sentences:
                if sentence[:4] in candidate:
                    vec[idx] += 1
        return vec


class Highlight(Text):
    def __init__(self, text):
        super().__init__(text)
        print("Highlight")
        wordrank_extractor = KRWordRank(min_count=3, max_length=10)
        self.keywords, rank, graph = wordrank_extractor.extract(self.paragraphs)
        self.path = "/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT"
        p = []
        kw = []
        for k, v in self.keywords.items():
            p.append(okt.pos(k))
            kw.append(k)
        words = self.text.split(' ')
        s = set()
        keylist = [word for i in kw for word in words if i in word]
        keylist = [i for i in keylist if len(i)>2]
        for i in keylist:
            if len(i)>2:
              s.add(i)
        # print("KEYLIST: ",keylist)

        p = [okt.pos(word) for word in s]
        self.s = set()
        for idx in range(len(p)):
            ls = p[idx]
            for tags in ls:
              word,tag = tags
              if tag == "Noun":
                if len(word)>=2:
                  self.s.add(word)
        self.keys = []
        for temp in self.s:
            self.keys.append(" " + str(temp))
        print("KEYWORDS: ", self.keys)

    def add_tags_conj(self, txt):
        conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록, 예를 들어, 반면에, 하지만, 그렇다면, 바로, 이에 대해'
        conj = conj.replace("'", "")
        self.candidates = conj.split(",")
        self.newtext = deepcopy(txt)
        self.idx = [(i, i + len(candidate)) for candidate in self.candidates for i in
                        self.findall(candidate, txt)]
        for i in range(len(self.idx)):
            try:
                self.idx = [(start, start + len(candidate)) for candidate in self.candidates for start in
                            self.findall(candidate, self.newtext)]
                word = self.newtext[self.idx[i][0]:self.idx[i][1]]
                self.newtext = word.join([self.newtext[:self.idx[i][0]], self.newtext[self.idx[i][1]:]])
            except:
                pass
        return self.newtext

In [ ]:
class Summarize(Highlight):
    def __init__(self, text, paragraph_no):
      super().__init__(text)
      print("length of paragraphs ",len(self.paragraphs))
      self.txt = self.paragraphs[paragraph_no]
      self.paragraph_no = paragraph_no

    def summarize(self):
        url = "https://api.smrzr.io/v1/summarize?num_sentences=5&algorithm=kmeans"
        headers = {
            'content-type': 'raw/text',
            'origin': 'https://smrzr.io',
            'referer': 'https://smrzr.io/',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-site',
            "user-agent": "Mozilla/5.0"
        }
        resp = requests.post(url, headers=headers, data= self.txt.encode('utf-8'))
        assert resp.status_code == 200
        summary = resp.json()['summary']
        temp = summary.split('\n')
        print("BERT: ", temp)
        return temp


    def summarizeTextRank(self):
        tr = TextRank(sent_tokenize)
        summary = tr.summarize(self.txt, num_sentences=5).split('\n')
        print("Textrank: ",summary)
        return summary


    def summarizeLexRank(self):
        lr = LexRank()
        lr.summarize(self.txt)
        summaries = lr.probe()
        print("Lexrank: ",summaries)
        return summaries

    def ensembleSummarize(self):
        a = np.array(self.countMatcher(self.summarize(), self.paragraph_no))
        
        try:
          b = np.array(self.countMatcher(self.summarizeLexRank(), self.paragraph_no))
        except:
          b = np.zeros_like(a)
        c = np.array(self.countMatcher(self.summarizeTextRank(),self.paragraph_no))
        result= a+b+c
        i, = np.where(result == max(result))
        txt, index = self.docs[self.paragraph_no][i[0]], i[0]
        return txt, index

In [ ]:
result = chked
high = Highlight(result)

TEXT
Highlight
KEYLIST:  ['그런데', '바이러스', '바이러스', '바이러스', '바이러스', '바이러스', '따라서', '따라서', '따라서', '따라서', '직업에', '이런데', '노출도', '노출도', '노출도', '백신을', '백신에', '백신의', '백신의', '백신의', '백신이나', '맞춰야', '맞춰야지', '맞춰야', '맞춰야', '그렇게', '그렇게', '그렇게', '그래서', '그래서', '그래서', '문제가', '문제가', '문제들이', '문제를', '문제가', '목적에', '목적에', '목적에', '특성상', '특성상', '특성뿐만이', '때문에', '때문에', '때문에', '그래가지고', '가지고', '받아가지고', '있어가지고', '가지고', '가지고', '어떻게', '어떻게', '어떻게', '알고리즘으로', '알고리즘을', '알고리즘', '얘기를', '얘기를', '하면은', '종사하면', '하면은', '다른데', '다른게', '있어가지고', '있어서', '있어.', '아니면은', '아니라', '아니고요.', '아니면은', '사람들아', '사람들이', '사람들이', '적용하고', '적용해', '적용할', '활동하는', '야외활동이라던가', '활동이', '활동량이', '생각하기로는', '생각했습니다.', '생각을', '인구가', '인구밀도', '인구의', '달라진다.', '달라질', '달라지기']
KEYWORDS:  [' 때문', ' 백신', ' 문제', ' 사람', ' 따라서', ' 가지', ' 특성', ' 알고리즘', ' 노출', ' 활동량', ' 목적', ' 다른', ' 종사', ' 적용', ' 직업', ' 얘기', ' 활동', ' 인구', ' 밀도', ' 바이러스', ' 야외', ' 생각']


In [ ]:
summarizer = Summarize(chked, 0)
sum, id = summarizer.ensembleSummarize()
print("summarized ",sum)

TEXT
Highlight
KEYLIST:  ['그런데', '바이러스', '바이러스', '바이러스', '바이러스', '바이러스', '따라서', '따라서', '따라서', '따라서', '직업에', '이런데', '노출도', '노출도', '노출도', '백신을', '백신에', '백신의', '백신의', '백신의', '백신이나', '맞춰야', '맞춰야지', '맞춰야', '맞춰야', '그렇게', '그렇게', '그렇게', '그래서', '그래서', '그래서', '문제가', '문제가', '문제들이', '문제를', '문제가', '목적에', '목적에', '목적에', '특성상', '특성상', '특성뿐만이', '때문에', '때문에', '때문에', '그래가지고', '가지고', '받아가지고', '있어가지고', '가지고', '가지고', '어떻게', '어떻게', '어떻게', '알고리즘으로', '알고리즘을', '알고리즘', '얘기를', '얘기를', '하면은', '종사하면', '하면은', '다른데', '다른게', '있어가지고', '있어서', '있어.', '아니면은', '아니라', '아니고요.', '아니면은', '사람들아', '사람들이', '사람들이', '적용하고', '적용해', '적용할', '활동하는', '야외활동이라던가', '활동이', '활동량이', '생각하기로는', '생각했습니다.', '생각을', '인구가', '인구밀도', '인구의', '달라진다.', '달라질', '달라지기']
KEYWORDS:  [' 때문', ' 백신', ' 문제', ' 사람', ' 따라서', ' 가지', ' 특성', ' 알고리즘', ' 노출', ' 활동량', ' 목적', ' 다른', ' 종사', ' 적용', ' 직업', ' 얘기', ' 활동', ' 인구', ' 밀도', ' 바이러스', ' 야외', ' 생각']
length of paragraphs  1
BERT:  ['그 알고리즘으로 막상 해 놨길래 했는데 거기서 그 의료인들이 거의 다 빠지고 1순위에서 의료인들이 제일 필요한데 개를 다 빠지고 다른데 접종을 하는 것 때문에

In [ ]:
sum

'그렇다면 직업 특성상 재택근무가 아예 불가능한 집중된 섞여 있을 거 같은데 직업 특성뿐만이 아니라 그냥 바이러스 노출도 제일 중요한 거 같기도 하고 그 그게 이제 목적에 따라서 누굴 먼저 맞춰야 되니까 조금 다른게 그 감염률을 나 죽고 싶으면 네 사실 뭐랄까 제일 활발하게 활동하는 새끼들을 먼저 맞춰야지 그래야지 지금 뭐 그 퍼트리고 다니는 애들이 안 걸려 있으니까 그래서 그런 경우 조금 야외활동이라던가 이런 것들이 활동이 많은 젊은 층들이 맞춰야 되는 반면에 치사율을 낮추고자를 목표로 하면은 우선은 노인 인구가 가장 큰 높으니까 애들 먼저 맞춰야 된다 이런 일이 있어가지고 달달 말씀이시죠 목적에 따라 달라진다.'

- 사용자 인식(speaker identification)이 됐으면 좋겠다 -- clova NOTE 사용시 해결

> 무료 api는 supervised만 있는 듯

>google speech api는 한국어 speaker diarization 지원 X

- 시간단위로 잘리는 것 루프 만들기

- 기본 웹프레임워크 만들기

- 아웃풋 어떤 모양일지?
